In [1]:
import os
with open("the-verdict.txt","r",encoding="utf-8") as f:
  raw_text=f.read()

print(len(raw_text))
print(raw_text[:10])

20480
I HAD alwa


NOW WE WILL CONVERT THIS TEXT TO individual words
using re library

In [2]:
import re

text="Hello, world. This, is a test"
result = re.split(r'(\s)',text) #seperating based on whitespaces
result=re.split(r'([,.]\s)',text) # seperate comma,fullstop,whitespace all
result = list(filter(str.strip, result))
# removing whitespaces but it can be sometime important
#eg. python code, but here no need and it would save memory

#we want all symobols also to be seperate tokesn ?/... and all
result = re.split(r'[\s"><./;|\-_*&^%$!`~]+', text)
result = list(filter(str.strip, result))
print(result)


['Hello,', 'world', 'This,', 'is', 'a', 'test']


In [3]:
preprocessed=re.split(r'[\s"><./;|\-_*&^%$!`~\[\]\(\)\{\}]+', raw_text)
preporcessed = list(filter(str.strip, preprocessed))
print(preporcessed[:20])

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', 'though', 'a', 'good', 'fellow', 'enough', 'so', 'it', 'was', 'no', 'great']


CONVERTING THE INDIVUDAL WORDS/TOKENS into IDs

In [4]:
#getting no of unqie words to assign id after sorting

preprocessed = re.split(
    r"""[\s'><./;|\-_*&^%$!`~\[\]\"(\)\{\}]+""",
    raw_text
)

preprocessed = [w.lower() for w in preprocessed if w]

unique_words = sorted(set(preprocessed))

print(len(unique_words))

print(list(unique_words)[:10])

1219
[':', '?', 'a', 'abdication', 'able', 'about', 'above', 'abruptly', 'absolute', 'absorbed']


In [5]:
# creating the vocabulary itself
vocab = {}
for i, word in enumerate(unique_words):
    vocab[word] = i

# enumerate(unique_words) gives you (index, value) pairs. here index,word

print(list(vocab.items())[:10])


[(':', 0), ('?', 1), ('a', 2), ('abdication', 3), ('able', 4), ('about', 5), ('above', 6), ('abruptly', 7), ('absolute', 8), ('absorbed', 9)]


Creating and ensembling  our custom tokenizer

In [6]:
class SimpleTokenizerV1:
  def __init__(self,vocab):
    self.str_to_int=vocab
    self.int_to_str={i: s for s,i in vocab.items()}#called as comprehensions

  def encode(self,Text):
    preprocessed = re.split(
        r"""[\s'><./;|\-_*&^%$!`~\[\]\"(\)\{\}]+""",
        Text
    )
    preprocessed=map(str.lower,preprocessed)
    preprocessed=filter(str.strip,preprocessed)

    #as map filter returns iterator, and they pass fxn so no () with strip,lower
    preprocessed=list(preprocessed)
    ids=[self.str_to_int[i] for i in preprocessed]
    return ids

  def decode(self,ids):
    text = " ".join(self.int_to_str[i] for i in ids)

    # add space after punctuation if missing
    text = re.sub(r"([.,!?;:])(?=\S)", r"\1 ", text)

    return text

NOW CHECKING IT


In [7]:
tokenizer = SimpleTokenizerV1(vocab)

In [8]:
Text="It was not till three years later that, in the course of a few weeks' idling on the Riviera, it suddenly occurred to me to wonder why Gisburn had given up his painting."

In [9]:
encoded=tokenizer.encode(Text)
print(encoded)

[551, 1149, 709, 1071, 1062, 1208, 579, 1040, 533, 1041, 212, 723, 2, 371, 1160, 530, 730, 1041, 870, 551, 999, 721, 1078, 646, 1078, 1197, 1181, 431, 463, 435, 1119, 508, 758]


In [10]:
decoded=tokenizer.decode(encoded)
print(decoded)

it was not till three years later that, in the course of a few weeks idling on the riviera, it suddenly occurred to me to wonder why gisburn had given up his painting


But, What if the word is not present

In [11]:
#using srijan
Text="It was not till three years later that, in the course of a few weeks' idling on the Riviera, it suddenly occurred to me to wonder why Srijan had given up his painting."

encoded=tokenizer.encode(Text)
print(encoded)

KeyError: 'srijan'

In [16]:
#so we got an oov(out of vocab error)
#TO FIX IT WE USE SPECIAL CONTEXT TOKENS (as a backup for our model <unk>)

vocab2 = vocab.copy()
vocab2["<UNK>"] = 9999

class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int=vocab
        self.int_to_str={i: s for s,i in vocab.items()}#called as comprehension

    def encode(self, text):
        # split text into words
        words = re.split(
            r"""[\s'><./;|\-_*&^%$!`~\[\]\"()\{\}]+""",
            text
        )

        ids = []

        for w in words:
            w = w.lower().strip()

            if w == "":
                continue

            if w in self.str_to_int:
                ids.append(self.str_to_int[w])
            else:
                ids.append(self.str_to_int["<UNK>"])

        return ids

    def decode(self, ids):
        words = []

        for i in ids:
            if i in self.int_to_str:
                words.append(self.int_to_str[i])
            else:
                words.append("<UNK>")

        return " ".join(words)

In [17]:
tokenizer = SimpleTokenizerV2(vocab2)

text = "It was not till three years later that, in the course of a few weeks' idling on the Riviera, it suddenly occurred to me to wonder why Srijan had given up his painting."
encoded = tokenizer.encode(text)
decoded = tokenizer.decode(encoded)

print("Encoded:", encoded)
print("Decoded:", decoded)

Encoded: [551, 1149, 709, 1071, 1062, 1208, 579, 1040, 533, 1041, 212, 723, 2, 371, 1160, 530, 730, 1041, 870, 551, 999, 721, 1078, 646, 1078, 1197, 1181, 9999, 463, 435, 1119, 508, 758]
Decoded: it was not till three years later that, in the course of a few weeks idling on the riviera, it suddenly occurred to me to wonder why <UNK> had given up his painting
